PRACTICAL TOOLS FOR PARSING DATA WITH PYTHON

USED PACKAGES:
urllib3                   2.2.1
requests                  2.31.0
openpyxl                  3.1.2
pandas                    2.2.2
numpy                     1.26.4
beautifulsoup4            4.12.2
lxml
MechanicalSoup            1.3.0

In [2]:
import requests as req
import numpy as np
import pandas as pd
from IPython.display import Image
from datetime import datetime
from bs4 import BeautifulSoup
import webbrowser
import mechanicalsoup
import json


I. Downloading data when a download link is available. This is the ideal case which would often require you not to use the code if you will download the data once from the website. I am still going to show you how to download and read the data using Python if such link is available in case you would have to download the data regularly.

STEP 1: Identify the link

Easiest ways to identify the link:
   - a) Go to source website and check for download button. Often download button will be given as image with the image describing the file type. Once you find download button, click right button on your mouse and select "copy link address"
  ![Example](images/locate_download_button.png)
   - b) Sometimes, download button is not directly leading to the main download URL, instead it acts as trigger  - you click on button, it redirects to the original URL. In this case, "copy link address" won't give you the correct address. Instead, you may locate downloaded file in the Downloads tab of Chrome browser (CTRL+J will take you there) and you can access correct download link using "copy link address" like in this example
   ![Example](images/locate_download_link.png)

STEP 2. Use the retrieved link to parse data on Python. From the link we can already understand that this is an xlsx file.

In [6]:
URL = "https://nbg.gov.ge/fm/%E1%83%A1%E1%83%A2%E1%83%90%E1%83%A2%E1%83%98%E1%83%A1%E1%83%A2%E1%83%98%E1%83%99%E1%83%90/monetary_statistics/eng/money-aggregates-and-monetary-ratioseng.xlsx?v=r5z4c"

filename = "data.xlsx"

# you may first save the file and load it using pandas library. However, saving first as a file may be useful if URL doesn't work and you receive some error info message in text/html form.
# By checking response, you may understand what is the issue and try to solve it.
response = req.get(URL)

# Check if data request was successful
if response.ok:
    # Save CSV response to a file
    with open(filename, 'wb') as f:
        f.write(response.content)
    print('Data have been saved successfully ')
else:
    print('Failed to retrieve data:', response.status_code)

df = pd.read_excel(filename, index_col=0, header=2)
df.head()


Data have been saved successfully 


,Currency in Circulation,Monetary Base,Money Aggregates (M3),Money Aggregates (M2),Money Aggregates (M1),Money Outside Banks (M0),Transferable Deposits in National Currency,Term Deposits in National Currency,Deposits in Foreign Currency,"Deposits, Total",...,Money Multiplier (M3),Money Multiplier (M2),Monetization Coefficient * (M3),Monetization Coefficient * (M2),Money Velocity * (M3),Money Velocity * (M2),Monetization Coefficient (M3),Monetization Coefficient (M2),Money Velocity (M3),Money Velocity (M2)
Period,,,,,,,,,,,,,,,,,,,,,
2024-03-31,5675.703,10669.549,43186.961,25403.360783,16501.021816,4676.592,11824.429816,8902.338967,17783.600316,38510.369099,...,4.047684,2.380922,49.294313,28.778976,2.028632,3.474759,50.006902,29.414975,1.999724,3.399629
NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaT,Source: The National Bank of Georgia.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaT,Notes: * Average from the beginning of the yea...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
URL = "https://nbg.gov.ge/fm/%E1%83%A1%E1%83%A2%E1%83%90%E1%83%A2%E1%83%98%E1%83%A1%E1%83%A2%E1%83%98%E1%83%99%E1%83%90/monetary_statistics/eng/money-aggregates-and-monetary-ratioseng.xlsx?v=r5z4c"
# Alternatively you can download data using URL directly with pandas
df = pd.read_excel(URL, index_col=0, header=2)
df.head()

,Currency in Circulation,Monetary Base,Money Aggregates (M3),Money Aggregates (M2),Money Aggregates (M1),Money Outside Banks (M0),Transferable Deposits in National Currency,Term Deposits in National Currency,Deposits in Foreign Currency,"Deposits, Total",...,Money Multiplier (M3),Money Multiplier (M2),Monetization Coefficient * (M3),Monetization Coefficient * (M2),Money Velocity * (M3),Money Velocity * (M2),Monetization Coefficient (M3),Monetization Coefficient (M2),Money Velocity (M3),Money Velocity (M2)
Period,,,,,,,,,,,,,,,,,,,,,
1995-10-31,94.955,120.284,131.419,112.337,107.279,84.708,22.571,5.059,19.081,46.710,...,1.092573,0.933937,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1995-11-30,107.529,133.659,145.258,126.224,121.151,98.254,22.897,5.073,19.034,47.004,...,1.086786,0.944378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1995-12-31,131.365,153.814,179.441,160.145,154.534,124.779,29.755,5.611,19.296,54.662,...,1.166613,1.041161,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996-01-31,129.294,150.614,169.911,149.923,144.440,120.331,24.109,5.483,19.988,49.580,...,1.128119,0.995410,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996-02-29,128.817,153.203,171.306,148.395,142.815,117.945,24.870,5.580,22.911,53.361,...,1.118168,0.968621,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df.tail()

,Currency in Circulation,Monetary Base,Money Aggregates (M3),Money Aggregates (M2),Money Aggregates (M1),Money Outside Banks (M0),Transferable Deposits in National Currency,Term Deposits in National Currency,Deposits in Foreign Currency,"Deposits, Total",...,Money Multiplier (M3),Money Multiplier (M2),Monetization Coefficient * (M3),Monetization Coefficient * (M2),Money Velocity * (M3),Money Velocity * (M2),Monetization Coefficient (M3),Monetization Coefficient (M2),Money Velocity (M3),Money Velocity (M2)
Period,,,,,,,,,,,,,,,,,,,,,
2024-03-31,5675.703,10669.549,43186.961,25403.360783,16501.021816,4676.592,11824.429816,8902.338967,17783.600316,38510.369099,...,4.047684,2.380922,49.294313,28.778976,2.028632,3.474759,50.006902,29.414975,1.999724,3.399629
NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaT,Source: The National Bank of Georgia.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaT,Notes: * Average from the beginning of the yea...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
df_cleaned = df.loc[df.index.notnull()]

In [15]:
df_cleaned.tail()

,Currency in Circulation,Monetary Base,Money Aggregates (M3),Money Aggregates (M2),Money Aggregates (M1),Money Outside Banks (M0),Transferable Deposits in National Currency,Term Deposits in National Currency,Deposits in Foreign Currency,"Deposits, Total",...,Money Multiplier (M3),Money Multiplier (M2),Monetization Coefficient * (M3),Monetization Coefficient * (M2),Money Velocity * (M3),Money Velocity * (M2),Monetization Coefficient (M3),Monetization Coefficient (M2),Money Velocity (M3),Money Velocity (M2)
Period,,,,,,,,,,,,,,,,,,,,,
2023-11-30,5505.67,11619.707,41362.975,23409.244627,15818.065013,4430.714,11387.351013,7591.179613,17953.729828,36932.260455,...,3.559726,2.014616,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-12-31,5934.138,12019.229,43192.256,25112.442448,17546.776915,4721.458,12825.318915,7565.665533,18079.813336,38470.797784,...,3.593596,2.089356,48.129532,27.080403,2.077726,3.692707,53.824758,31.294293,1.857881,3.195471
2024-01-31,5520.712,10770.450,42163.486,24466.210001,16009.099317,4519.537,11489.562317,8457.110684,17697.276152,37643.949153,...,3.914738,2.271605,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-02-29,5504.812,10572.330,42364.218,24692.726571,15935.265875,4475.604,11459.661875,8757.460696,17671.491691,37888.614261,...,4.007084,2.335599,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-03-31,5675.703,10669.549,43186.961,25403.360783,16501.021816,4676.592,11824.429816,8902.338967,17783.600316,38510.369099,...,4.047684,2.380922,49.294313,28.778976,2.028632,3.474759,50.006902,29.414975,1.999724,3.399629


II. Sometimes API/URL will let you choose additional options. For example, the following data from Kazakhstan Stock Exchange can be downloaded for specific date range.
https://kase.kz/en/money_market/repo-indicators/tonia/archive-xls/22.05.2024/22.05.2024/

In [19]:
# download data from 2010 till current date
from_date = "01.01.2010"

today_date = datetime.today().strftime("%d.%m.%Y")
print(today_date)


22.05.2024


In [23]:
base_url = "https://kase.kz/en/money_market/repo-indicators/tonia/archive-xls/"

req_url = f"{base_url}{from_date}/{today_date}/"

df_2 = pd.read_excel(req_url, index_col=0, header=1)
df_2.head()

,Close,"Transaction volume, bn KZT","Transactions amount, m USD"
Date,,,
05.01.10,0.19,20.50,NaN
06.01.10,0.13,12.44,NaN
10.01.10,0.19,9.30,NaN
11.01.10,0.14,16.43,NaN
12.01.10,0.14,6.83,NaN


In [24]:
df_2.tail()

,Close,"Transaction volume, bn KZT","Transactions amount, m USD"
Date,,,
16.05.24,15.32,417.78,946.46
17.05.24,15.13,653.40,1479.68
20.05.24,13.87,710.21,1601.59
21.05.24,13.08,597.05,1345.55
22.05.24,13.04,571.90,1294.47


III. In many cases, there won't be a download link, instead data will be given as html table. Downloading such data is not as difficult as it seems. One simply need to locate the table inside html structure by using beautifulsoup package and then put it into desired format. The following data belongs to Bangladesh Interbank market. When users click on the following link https://www.bb.org.bd/en/index.php/monetaryactivity/call_money_market, the webpage with data on interbank market data of latest business day opens up. As a first step, we will only download data for the current day, but I will show you below how the historical data can be downloaded as well. Additionally, you may see that there is no download link available, thus we have to extract data from html


Step 1: Locate the table first in html structure. You may make URL request with entire webpage and read the raw response, but it is a tedious approach. Instead we will use Chrome console.

Press F12 on your keyboard or right click on your mouse and select Inspect. You will see a new pane opening on the right. Click once on inspector tool and then click an area on the table like shown in the picture

![Example](images/locate_table_console.png)

This will take you to the place where select item is located in entire web html structure. HTML table is designed in hiearchical structure, go either higher or lower in structure until you think that segment covers entire information you need about the table. In most cases, item class will be called table and items in lower hiearchy of the table will have tags as thead and tbody. thead tag refers to the headers of the table, while tbody will contain data. Below is the shown the segment I choose for this example. You may see that whenever I hover my mouse on right pane on an item in html structure, the item is highlighted as well as area it corresponds in the left pane.
![Example](images/locate_table_html.png)



Step 2: Use the attributes of table to locate it using beautiful soup and later extract the date. The task here is to help beautiful soup to locate and select part of html which we can see visually on the browser. As you can see, there are two potential characteristics we can use - classname ("table-responsive") and tag ("table") to locate table. ID would be most ideal, since it would help us uniquely identify table, however, developers of this website have not provided it. Instead we will use classname.In case, there are more than 1 table with such class, we will simply use debugger to see which one is right table.


In [5]:
req_url = "https://www.bb.org.bd/en/index.php/monetaryactivity/call_money_market"
response = req.get(req_url)
# Check if data request was successful
if response.ok:
    # Save CSV response to a file
    soup = BeautifulSoup(response.content.decode("utf-8"))
    print('Data have been saved successfully ')
else:
    print('Failed to retrieve data:', response.status_code)



Data have been saved successfully 


In [7]:
#use table tag to locate table
tables = soup.find_all("table")
print(len(tables))

1


In [6]:
# use classname attribute
# you may use other attributes in this way if they are available
tables = soup.find_all(attrs={"class": "table-responsive"})
print(len(tables))

1


In [8]:
#this is another way to find by class
tables = soup.find_all("div", class_="table-responsive")
print(len(tables))

1


In [9]:
# select the first table since we have only 1 of them
table = tables[0]

In [25]:
headers_raw = table.select_one("thead")
t_content = table.select_one("tbody")

In [34]:
# Find all <th> elements within <thead>
header_cells = headers_raw.find_all("th")

# Extract the text from each <th> element
headers = [th.get_text(strip=True) for th in header_cells]
# this is not the ideal way, but we have to rearrange headers since headers are given in two rows
headers.remove("Interest rate(%)")
headers.remove("Number of Deals")
headers.append("Number of Deals")
print(headers)
# alternatively you can simply write down column names rather than extract it from html, then make sure the html contains at least your column names

['Product', 'Maturity', 'Amount(Crore Taka)', 'Highest', 'Lowest', 'Average', 'Number of Deals']


In [43]:
# Find all rows in tbody
rows = t_content.find_all("tr")

# Extract data from each row
data = []
for row in rows:
    cells = row.find_all("td")
    if len(cells)==1:
        # skip the first row
        continue
    cell_texts = [cell.get_text(strip=True) for cell in cells]
    data.append(cell_texts)
print(data)

[['Overnight', '1 Day(/s)', '5270.60', '10.00', '8.50', '8.97', '64'], ['Short Notice', '2 Day(/s)', '20.00', '9.50', '9.50', '9.50', '1'], ['Short Notice', '5 Day(/s)', '80.00', '11.00', '11.00', '11.00', '1'], ['Short Notice', '7 Day(/s)', '182.00', '11.00', '10.90', '10.99', '4'], ['Short Notice', '8 Day(/s)', '20.00', '11.90', '11.90', '11.90', '1'], ['Term', '90 Day(/s)', '100.00', '12.75', '12.75', '12.75', '1']]


In [44]:
# Create a pandas DataFrame
df_3 = pd.DataFrame(data, columns=headers)

In [45]:
df_3.head()

,Product,Maturity,Amount(Crore Taka),Highest,Lowest,Average,Number of Deals
0,Overnight,1 Day(/s),5270.60,10.00,8.50,8.97,64
1,Short Notice,2 Day(/s),20.00,9.50,9.50,9.50,1
2,Short Notice,5 Day(/s),80.00,11.00,11.00,11.00,1
3,Short Notice,7 Day(/s),182.00,11.00,10.90,10.99,4
4,Short Notice,8 Day(/s),20.00,11.90,11.90,11.90,1


In [47]:
#Alternative way to parse html using pandas
df_tables = pd.read_html(req_url)

In [48]:
#this will create list for each table in html, in our case, there is only one table
len(df_tables)

1

In [52]:
# select the table
df_4 = df_tables[0]

In [54]:
df_4.head()

Product       Maturity Amount (Crore Taka) Interest rate(%)  \
         Product       Maturity Amount (Crore Taka)          Highest   
0  05 June, 2024  05 June, 2024       05 June, 2024    05 June, 2024   
1      Overnight      1 Day(/s)              245.00             9.95   
2   Short Notice     14 Day(/s)              100.00            10.00   

                                Number of Deals  
          Lowest        Average Number of Deals  
0  05 June, 2024  05 June, 2024   05 June, 2024  
1           8.50           8.81               8  
2           9.50           9.75               2

In [59]:
df_3.head()

,Product,Maturity,Amount(Crore Taka),Highest,Lowest,Average,Number of Deals
0,Overnight,1 Day(/s),5270.60,10.00,8.50,8.97,64
1,Short Notice,2 Day(/s),20.00,9.50,9.50,9.50,1
2,Short Notice,5 Day(/s),80.00,11.00,11.00,11.00,1
3,Short Notice,7 Day(/s),182.00,11.00,10.90,10.99,4
4,Short Notice,8 Day(/s),20.00,11.90,11.90,11.90,1


In [ ]:
# pandas fails to get all the data
len(df_4)


In [58]:
len(df_3)

6

IV. It is not always straightforward to download data, even using html. In many cases, often using URL alone will not be enough. Website will require you to send additional details in the payload to make it look like an authentic request using browser. Chrome and other browsers do it on the backend, however, if you can open it in the browser, there must be a way to open it using a code as well. Again we will use Chrome console to figure out what is required.


Continuing with above example, now we need to download the data for history as well. We need to define what additional attributes we should send along the URL request to get full data. The process is a bit complicated and not everything can be covered with one example. Often you may require to get creative to identify source. Easiest way is to use Chrome console.

Step 1: Enter Chrome console by using Inspect or pressing F12 and then go to Network. Then on left pane, select dates to show the full data. In Network, full list of URl requests executed in the backend is listed. Here you need to locate which request is calling the table data. That is the tricky part. Sometimes there are plenty of requests that you may actually spend a lot of time for it. This example is rather easier to find. You may see in the image below that request is called call_money_market and includes link for the website. This is the one we look for.![Example](images/locate_url_request.png)

Step 2: We need to identify the attributes. First thing we must identify is how to set dates. There are two places we check. Request headers and payload. Not all the attributes written here are required to get data. Instead we should start with the things we must add and gradually add the rest of the attributes until we get html page with table. In this example, date is set using date_picker attribute in payload. We also send user-agent to identify session as a browser session. You can also see it in request headers. I ended up using following attributes ![Example](images/req_attributes.png)

In [86]:
# check with local browser to see if you have retrieved the correct html
webbrowser.open("temp_file.html")

True

In [90]:
# extract data
tables = soup.find_all(attrs={"class": "table-responsive"})
print(len(tables))

1


In [91]:
# select the first table since we have only 1 of them
table = tables[0]
headers_raw = table.select_one("thead")
t_content = table.select_one("tbody")
# Find all <th> elements within <thead>
header_cells = headers_raw.find_all("th")

# Extract the text from each <th> element
headers = [th.get_text(strip=True) for th in header_cells]
# this is not the ideal way, but we have to rearrange headers since headers are given in two rows
headers.remove("Interest rate(%)")
headers.remove("Number of Deals")
headers.append("Number of Deals")
print(headers)

['Product', 'Maturity', 'Amount(Crore Taka)', 'Highest', 'Lowest', 'Average', 'Number of Deals']


In [114]:
# Find all rows in tbody
rows = t_content.find_all("tr")

# Extract data from each row
data = []
dates = []
current_date = None
for row in rows:
    cells = row.find_all("td")
    if len(cells)==1:
        current_date = cells[0].get_text(strip=True)
        continue
    cell_texts = [cell.get_text(strip=True) for cell in cells]
    data.append(cell_texts)
    #for each date, there are different types of maturity, thus we need to repeat each date by number of maturity types
    dates.append(current_date)

In [ ]:
print(dates)

In [116]:
# convert dates to pandas datetime format
date_index = pd.to_datetime(dates, format='%d %b, %Y')

In [117]:
df_5 = pd.DataFrame(data, columns=headers, index=date_index)

In [119]:
df_5.head()

,Product,Maturity,Amount(Crore Taka),Highest,Lowest,Average,Number of Deals
2024-05-30,Overnight,1 Day(/s),3477.66,10.00,8.50,9.12,47
2024-05-30,Short Notice,7 Day(/s),134.00,11.25,10.00,11.05,5
2024-05-30,Short Notice,14 Day(/s),64.80,12.00,10.00,10.89,3
2024-05-30,Term,33 Day(/s),9.00,12.00,12.00,12.00,1
2024-05-30,Term,91 Day(/s),2.00,12.55,12.55,12.55,1


V. There are cases that you have to sign-in in order to access data. If such credentials are required, then it is not possible to simply use URL and get the data. If username and password can be set using attributes, you can use a similiar procedure to the example above and do it. However, another and probably more user-friendly approach is to mimic browser behaviour almost as if you are using your keyboard and mouse to interact with it. In python, mechanicalsoup allows us to fill-in signin forms and make further URL requests in a signed-in type of session. I use Mechanical soup here but has its limitations. Java has selenium packages which is a more advanced tool, however, it is outside scope of this tutorial. Instead I will show far simpler solution in case VI for single-use cases.


In this example, we are going to sign-in to IMF. We will sign in first and then download the data using Python code. If the data is not in excel format, but an html table, which is when you will mostly need to use the code to parse data, you can stil use the same steps to sign in and do the rest using the examples about parsing from html above.

STEP 1 - Set username and password. You can write down in the code if you are the only user and you won't share the code. Since I am sharing the code with you, I am going to access it from local file. And later enter login page using mechanical soup browser

In [2]:
with open('user_data.json', 'r') as file:
    # Step 3: Load the JSON data
    data = json.load(file)

# Step 4: Access the attributes
username = data['username']
password = data['password']

In [85]:
req_url = "https://www.bb.org.bd/en/index.php/monetaryactivity/call_money_market"
# payload = {'date_picker': '01/05/2024 - 31/05/2024'} this format doesn't work

data = 'date_picker=01/05/2024 - 31/05/2024'

# Define headers with a User-Agent for Chrome
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36',
    'Content-Type': 'application/x-www-form-urlencoded'
}

response = req.post(req_url, data=data, headers=headers)

# Check if data request was successful
if response.ok:
    # Save CSV response to a file
    soup = BeautifulSoup(response.content.decode("utf-8"))
        # Create a temporary file
    with open("temp_file.html", 'w', encoding='utf-8') as temp_file:
        temp_file.write(response.text)
    print('Data have been saved successfully ')
else:
    print('Failed to retrieve data:', response.status_code)



Data have been saved successfully 


Step 2. Submit your credentials using login form and sign in. We first need to locate how the form is called inside html structure. Use Chrome console again to locate table in the html and check the tag of form. Luckily they are using default name "form"

In [3]:
browser = mechanicalsoup.StatefulBrowser(user_agent='MechanicalSoup')
browser.open("https://www.bookstore.imf.org/authgatewaylogin")



<Response [200]>

In [4]:
form = browser.select_form('form')

In [5]:
form.print_summary()

<input autocomplete="off" name="authenticity_token" type="hidden" value="R70m39KTfvMCO7yKC36bCkFEegbQkwoqHfRdGJSUrl4h42kMZRJhFvoiN9GGYjQj1zI2weVKPZtkVOj0kbLm8Q"/>
<input autocomplete="'off'" class="form-control" id="username" name="username" placeholder="Su dirección de correo electrónico es su nombre de usuario" type="text"/>
<input autocomplete="'off'" class="form-control large" id="password" name="password" placeholder="Contraseña" type="password" value=""/>
<input autocomplete="off" id="ssortn" name="ssortn" type="hidden" value="https://www.bookstore.imf.org"/>
<input autocomplete="off" id="commit" name="commit" type="hidden" value="1"/>
<button class="imf-button" id="update" name="button" type="submit">Enviar</button>


In [6]:
form.set("username", username)
form.set("password", password)

In [7]:
browser.submit_selected()

<Response [200]>

In [8]:
browser.launch_browser()

VI. Finally, in case you have a webpage that requires sign-in and is very complicated or impossible to handle with mechanicalsoup but you still want to parse data from webpage, this is not the ideal but a far simpler solution. Chrome and other browsers allow you to download webpage data. Among the downloaded data, you can find raw html file that includes all the information you need. Instead of making a URL request, we will use that file to extract data.

STEP 1. Click CTRL+S (CMD+S for MAC) or right click on an empty space on the webpage and select Save as. Then select folder you want to save. We also need to save complete page. Click save and wait for download to finish

STEP 2. Go to the folder with downloaded files. You will have one html file with the name you have selected and another folder with the same name. As long as data you want to access is stored in html, you can simply use html file to extract data. Often you may notice that html file itself is 1kb and contains no data, then search html files inside the folder and you will find another html file with bigger size. It may be called resource_content, raw_content etc. Just take the correct html file and nothing else, rename if you want
STEP 3. Go to python and use the code below to extract the data. HTML extraction is the same after you load html content to beautifulsoup (the method you use only depends on the content). THe only difference is how to load that content to beautiful soup.

# Example: https://www.nbkr.kg/index1.jsp?item=120&lang=ENG
Go to this webpage and save html page. The data is in the file, you don't need to check inside folder.

In [3]:
with open("source_html_file.html", encoding="utf-8") as fp:
    soup = BeautifulSoup(fp, 'html.parser')


In [6]:
# extract data
tables = soup.find_all(attrs={"class": "table content-table table-striped table-hover table-condensed table-bordered"})
print(len(tables))

1


In [9]:
# select the first table since we have only 1 of them
table = tables[0]
t_content = table.select_one("tbody")
tmp = t_content.find_all("tr")
headers_raw = tmp[0]
# Find all <th> elements within <thead>
header_cells = headers_raw.find_all("td")

# Extract the text from each <th> element
headers = [th.get_text(strip=True) for th in header_cells]
print(headers)

['Date', 'Registration number', 'Total offer volume(thou. soms)', 'Demand(thou. soms)', 'Sale(thou. soms)', 'Min yieldof accepted bids (%)', 'Max yieldof accepted bids (%)', 'Weighted averageyield of acceptedbids (%)']


In [10]:
# Find all rows in tbody
rows = tmp[1:-1]

# Extract data from each row
data = []
for row in rows:
    cells = row.find_all("td")
    cell_texts = [cell.get_text(strip=True) for cell in cells]
    data.append(cell_texts)

In [14]:
# convert dates to pandas datetime format
df_6 = pd.DataFrame(data, columns=headers)

In [15]:
df_6.head()

,Date,Registration number,Total offer volume(thou. soms),Demand(thou. soms),Sale(thou. soms),Min yieldof accepted bids (%),Max yieldof accepted bids (%),Weighted averageyield of acceptedbids (%)
0,11.06.2024,BD001240619,10 000 000.00,36 571 650.00,10 000 000.00,6.00,6.30,6.05
1,04.06.2024,BD001240612,10 000 000.00,27 450 850.00,10 000 000.00,6.30,7.20,6.58
2,28.05.2024,BD001240605,7 000 000.00,9 143 540.00,7 000 000.00,6.80,8.20,7.99
3,21.05.2024,BD001240529,7 000 000.00,13 716 020.00,7 000 000.00,9.00,10.00,9.82
4,14.05.2024,BD001240522,8 000 000.00,11 312 550.00,8 000 000.00,9.50,10.40,9.99


VI. Parsing text

Logic is the same as parsing tables. Just select the location you want to get text and use get_text method

# Example: https://www.theguardian.com/business/article/2024/jun/16/recovery-and-interest-rate-cuts-wont-be-enough-to-win-sunak-the-election

In [16]:
# let's get only the news title and text from this webpage
# specifically select an area in html and parse text from that area only
req_url = "https://www.theguardian.com/business/article/2024/jun/16/recovery-and-interest-rate-cuts-wont-be-enough-to-win-sunak-the-election"
response = req.get(req_url)

# Check if data request was successful
if response.ok:
    # Save CSV response to a file
    soup = BeautifulSoup(response.content.decode("utf-8"))
    print('Data have been saved successfully ')
else:
    print('Failed to retrieve data:', response.status_code)


Data have been saved successfully 


In [21]:
news_text = soup.find(id="maincontent").get_text()

In [23]:
print(news_text)

As the weeks roll by, Rishi Sunak’s decision to call the election before he needed to appears ever more curious. Unemployment is up and growth has stalled. NHS waiting lists have increased. There will be better news from this week’s annual inflation figures but it won’t make a difference to voting intentions.The case for holding on until the autumn was that it would give time for the Bank of England to start cutting interest rates and for recovery to become more firmly embedded. That case now looks all the stronger. Threadneedle Street is not going to deliver a pre-election cut in interest rates this week and by the time it does start to reduce the cost of borrowing, the Conservatives will be long gone.Likewise, when the growth figures for the second quarter come out in August they are likely to show solid – if unspectacular growth – of about 0.4-0.5%. After the 0.6% growth in the first quarter, it would have made Sunak’s argument that the economy has turned a corner that much stronger

In [55]:
# let's now try again with more careful approach and remove ads
# In inspection, you will see that all the news text under maincontent has tag p
texts = soup.find(id="maincontent").find("div", class_="article-body-commercial-selector").find_all("p", recursive=False)


In [56]:
parsed_text = []
for text in texts:
    parsed_text.append(text.get_text())
news_text_2 = "\n".join(parsed_text)

In [57]:
print(news_text_2)

As the weeks roll by, Rishi Sunak’s decision to call the election before he needed to appears ever more curious. Unemployment is up and growth has stalled. NHS waiting lists have increased. There will be better news from this week’s annual inflation figures but it won’t make a difference to voting intentions.
The case for holding on until the autumn was that it would give time for the Bank of England to start cutting interest rates and for recovery to become more firmly embedded. That case now looks all the stronger. Threadneedle Street is not going to deliver a pre-election cut in interest rates this week and by the time it does start to reduce the cost of borrowing, the Conservatives will be long gone.
Likewise, when the growth figures for the second quarter come out in August they are likely to show solid – if unspectacular growth – of about 0.4-0.5%. After the 0.6% growth in the first quarter, it would have made Sunak’s argument that the economy has turned a corner that much strong

Exercises

# 1. Extract data from the excel file in the following URL: https://www.bou.or.ug/bou/bouwebsite/bouwebsitecontent/statistics/InterestRates/Interest_rates.xlsx

In [ ]:
# your code here

# 2. Extract data about 1D interbank rates from this website: https://www.bcu.gub.uy/Politica-Economica-y-Mercados/Paginas/Tasa-1-Dia.aspx

<h>BONUS Content: A problem requiring nested solution<h>

We need to parse data about lobby groups. However, information is not provided under one table, one has to enter dedicated page for each of them to access full data. Luckily, we can use nested solution to get the data
# What you type in browser
https://transparency-register.europa.eu/searchregister-or-update/search-register_en
# Main URL that actually contains data. It is tricky to find this URL. When you enter the link above in browser, check the Network in console and
# you will see many other URL requests, one of them includes the link below. And that link has the table we want. You may see Responses of each URL to decide on the righ one.
https://ec.europa.eu/transparencyregister/public/alphabetical/REGISTRANTS/LATIN/a/1?lang=en
# second URL to get info on each lobby group by putting relevant ID for them
# again what you see on website
https://transparency-register.europa.eu/search-details_en?id=<lobby_id>
# url actually gets data on backend
https://ec.europa.eu/transparencyregister/public/PUBLIC/ORGANISATION/<lobby_id>?lang=en

In [4]:
# get list of IDs
page_number = 0 # starts from zero
req_url = f"https://ec.europa.eu/transparencyregister/public/alphabetical/REGISTRANTS/LATIN/a/{page_number}?lang=en"
# Define the minimum required headers
response = req.get(req_url)

# Check if data request was successful
if response.ok:
    # Save CSV response to a file
    soup = BeautifulSoup(response.content.decode("utf-8"))
    print('Data have been saved successfully ')
    # Create a temporary file
    with open("temp_file.html", 'w', encoding='utf-8') as temp_file:
        temp_file.write(response.text)
else:
    print('Failed to retrieve data:', response.status_code)

Data have been saved successfully 


In [5]:
tables = soup.find_all("table", class_="ecl-table ecl-table--zebra")
print(len(tables))

1


In [6]:
# select the first table since we have only 1 of them
table = tables[0]
headers_raw = table.select_one("thead")
t_content = table.select_one("tbody")
# Find all <th> elements within <thead>
header_cells = headers_raw.find_all("th")
# Extract the text from each <th> element
headers = [th.get_text(strip=True) for th in header_cells]
print(headers)

['Organisation’s identification number', 'Organisation name']


In [7]:
# Extract data from each row
data = []
rows = t_content.find_all("tr")
for row in rows:
    cells = row.find_all("td")
    cell_texts = [cell.get_text(strip=True) for cell in cells]
    data.append(cell_texts)

In [8]:
lobby_list = pd.DataFrame(data, columns=headers)

In [9]:
lobby_list.tail()

,Organisation’s identification number,Organisation name
5,138719717404-80,A. Menarini Industrie Farmaceutiche Riunite s....
6,185215246862-61,A.G.M.E.N. F.V.G. - A.P.S.
7,962348443419-15,A.I.A.D.
8,680443918500-51,A.P. Møller - Mærsk A/S
9,172031830923-23,A.S.T. s.r.l.


In [10]:
# now let's use IDs to get rest of data
id_list = list(lobby_list.iloc[:,0])

In [33]:
# this is a bulk solution to get all data. If some sub-tables have weird format, and you need those data, you shold treat each table separately,
# instead of looping through each table and applying same solution.
combined_lobby_data = dict()
for _id in id_list:
    lob_url = f"https://ec.europa.eu/transparencyregister/public/PUBLIC/ORGANISATION/{_id}?lang=en"
    response = req.get(lob_url)
    soup = BeautifulSoup(response.content.decode("utf-8"))
    tables = soup.find_all("table", class_="ecl-table ecl-table--zebra")
    data = dict()
    for table in tables:
        rows = table.find_all("tr")
        for row in rows:
            cells = row.find_all("td")
            # second column contains data. we could simply iterate over all tds, however, missing data will have no td tag and thus cause mismatch ibetween header and data
            if len(cells)>1:
                key = cells[0].get_text(strip=True)
                value = cells[1].get_text(strip=True)
                data[key] = value
    combined_lobby_data[_id] = data


In [34]:
df_7 = pd.DataFrame.from_dict(combined_lobby_data, orient='index').reset_index()


In [35]:
df_7.head()

,index,Organisation name:,REG Number:,Status:,Registration date:,The registrant performed the last (partial or annual) update on:,Next annual update due latest on:,Acronym:,Form of entity:,Website:,...,Main sources of funding in the most recent closed financial year:,Comment:,Contributions in the closed financial year (above €10 000 and 10% of the total budget):,Erietta Kourkoulou - Latsi,Total budget:,Complementary information if €0 budget:,SurnameFirst nameStart dateEnd dateCRAENHans28/06/202327/06/2024,CRAEN,Ministero del lavoro e delle politiche sociali,REISENHOFER EDOARDO
0,659985144655-48,"A Place for Rover, Inc.",659985144655-48,Activated,15/11/2021 17:28:16,03/11/2023 18:32:44,03/11/2024,N/A,Inc.,rover.com,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,777021553052-19,A Promise to Animals Non Profit Civil Law Society,777021553052-19,Activated,14/03/2024 15:03:30,N/A,01/03/2025,APA,Non profit Greek civil law society,https://www.apromisetoanimals.org/eng,...,DonationsMember's contributions,N/A,Contributor nameAmountErietta Kourkoulou - Lat...,"€168,000","€170,000",N/A,NaN,NaN,NaN,NaN
2,371263033273-46,A Rocha International,371263033273-46,Activated,21/11/2018 13:07:07,21/02/2024 14:31:13,18/01/2025,ARI,charity,http://arocha.org,...,DonationsGrants,N/A,N/A,NaN,"€1,688,321",N/A,NaN,NaN,NaN,NaN
3,192905850222-58,A-Gas International Ltd.,192905850222-58,Activated,22/05/2023 16:40:22,15/05/2024 09:57:51,15/05/2025,N/A,A-Gas is a private limited company with headqu...,www.agas.com,...,NaN,NaN,NaN,NaN,NaN,NaN,CRAEN,Hans,NaN,NaN
4,692018449211-82,A-Insinöörit Oy,692018449211-82,Activated,27/02/2023 17:35:45,06/02/2024 09:14:04,06/02/2025,AINS,Osakeyhtiö,https://www.ains.fi/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
df_7.to_excel("lobby_data.xlsx")